# Experiment 2 – Simulate Key Management Service + RBAC

**Goal:** Simulate Key Management Service (KMS) and Role-Based Access Control (RBAC) to manage access to encrypted data.

In this experiment, we will:

1. Define roles and their associated keys.
2. Implement functions to decrypt salary and password columns based on user roles.
3. Simulate the KMS and RBAC logic to demonstrate how access is controlled.

In [10]:
# Imports
import os
from dotenv import load_dotenv
from constants import BUCKET_NAME, SALARY_ENCRYPTED_FILE_NAME, ENGINEERING, HR, ADMIN
from cryptography.fernet import Fernet
from io import BytesIO
import boto3
import pandas as pd

In [11]:
# Load environment variables
load_dotenv()
FERNET_KEY = os.getenv("FERNET_KEY")
ADMIN_KEY = os.getenv("ADMIN_KEY")

In [12]:
def decrypt_salary(encrypted_column):
    """Real decrypt (KMS primitive). Only this function knows the key."""
    decrypted_bytes = Fernet(FERNET_KEY).decrypt(encrypted_column.encode())
    return int(decrypted_bytes.decode())

def get_key(role):
    if role == HR:
        return {
            "fernet_key": FERNET_KEY
        } 
    elif role == ADMIN:
        return {
            "fernet_key": FERNET_KEY,
            "admin_key": ADMIN_KEY
        }
    elif role == ENGINEERING:
        return None  
    else:
        return None
    
def retrieve_data(bucket_name, file_name):
    """Read a Parquet file from S3"""
    s3 = boto3.client("s3")
    response = s3.get_object(Bucket=bucket_name, Key=file_name)
    data = BytesIO(response['Body'].read())
    df = pd.read_parquet(data, engine="pyarrow")
    print(f"\n successfully loaded '{file_name}' into DataFrame.")
    return df

def decrypt_salary_with_key(key, file, encrypted_column):
    if key is None:
        print("No key provided, cannot decrypt.")
        return file
    fernet = Fernet(key)
    df = file.copy()
    df[encrypted_column] = df[encrypted_column].apply(
        lambda s: int(fernet.decrypt(s.encode()).decode())
    )
    return df

def decrypt_password_with_key(key, file, encrypted_column):
    if key is None:
        print("No key provided, cannot decrypt.")
        return file
    fernet = Fernet(key)
    df = file.copy()
    df[encrypted_column] = df[encrypted_column].apply(
        lambda s: str(fernet.decrypt(s.encode()).decode())
    )
    return df


def simulate_kms(df, encrypted_column, role):
    if role == HR:
        df = df.copy()
        df[encrypted_column] = df[encrypted_column].apply(
            lambda s: decrypt_salary(s)
        )
        return df
    else:
        return df

In [13]:

def simulate_kms_rbac(role):
    key = get_key(role)
    print("Retrieved key for role:", role)
    file = retrieve_data(BUCKET_NAME, SALARY_ENCRYPTED_FILE_NAME)
    if role == HR:
        df = decrypt_salary_with_key(key["fernet_key"], file, "Salary")
        return df
    elif role == ADMIN:
        df = decrypt_salary_with_key(key["fernet_key"], file, "Salary")
        df = decrypt_password_with_key(key["admin_key"], df, "Password")
        return df
    else:
        print("No decryption performed for role:", role)
        return file

# Simulate KMS and RBAC for different roles
hr_data = simulate_kms_rbac(HR)
print("HR Data:")
display(hr_data.head())

admin_data = simulate_kms_rbac(ADMIN)
print("Admin Data:")
display(admin_data.head())

engineering_data = simulate_kms_rbac(ENGINEERING)
print("Engineering Data (No decryption):")
display(engineering_data.head())

Retrieved key for role: RBAC_IN_DATA_LAKES_ROLE_SECURE_ANALYST

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.
HR Data:


,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,55000,gAAAAABpLrCMtGSy6lBrX4Ge4B_tADTvGpQkWCZ9MzGOi2...
1,2,Bob,bob@example.com,Engineering,72000,gAAAAABpLrCMqaffpffNlnWpyXigdcRQs35FD0evnJoPoV...
2,3,Charlie,charlie@example.com,Marketing,63000,gAAAAABpLrCMIeIebdmsd7i0hXysvI9OfMakQ_aSQys72G...
3,4,David,david@example.com,Finance,80000,gAAAAABpLrCMtj98pR40ir08dPPe10vTiJcJMu8eVpDDLy...
4,5,Eva,eva@example.com,Engineering,75000,gAAAAABpLrCMau4X6IX1fNkwVhznV7ncx_dOuB7sQqq2Px...


Retrieved key for role: RBAC_IN_DATA_LAKES_ROLE_ADMIN

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.
No key provided, cannot decrypt.
Admin Data:


,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,55000,gAAAAABpLrCMtGSy6lBrX4Ge4B_tADTvGpQkWCZ9MzGOi2...
1,2,Bob,bob@example.com,Engineering,72000,gAAAAABpLrCMqaffpffNlnWpyXigdcRQs35FD0evnJoPoV...
2,3,Charlie,charlie@example.com,Marketing,63000,gAAAAABpLrCMIeIebdmsd7i0hXysvI9OfMakQ_aSQys72G...
3,4,David,david@example.com,Finance,80000,gAAAAABpLrCMtj98pR40ir08dPPe10vTiJcJMu8eVpDDLy...
4,5,Eva,eva@example.com,Engineering,75000,gAAAAABpLrCMau4X6IX1fNkwVhznV7ncx_dOuB7sQqq2Px...


Retrieved key for role: RBAC_IN_DATA_LAKES_ROLE_READ_ONLY

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.
No decryption performed for role: RBAC_IN_DATA_LAKES_ROLE_READ_ONLY
Engineering Data (No decryption):


,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,gAAAAABpLrCMmHguX0r0xqGo92_ACuSCBKuA5vtWeQzLWH...,gAAAAABpLrCMtGSy6lBrX4Ge4B_tADTvGpQkWCZ9MzGOi2...
1,2,Bob,bob@example.com,Engineering,gAAAAABpLrCMHnO3jeaL2o1_rUru3c-R75ZcIIwMbcnFO9...,gAAAAABpLrCMqaffpffNlnWpyXigdcRQs35FD0evnJoPoV...
2,3,Charlie,charlie@example.com,Marketing,gAAAAABpLrCMRUQBH41jbiG0bFtmzXordV8vZvPXuHjkhs...,gAAAAABpLrCMIeIebdmsd7i0hXysvI9OfMakQ_aSQys72G...
3,4,David,david@example.com,Finance,gAAAAABpLrCM4S53NvA6aQpgA8Z1d17dAIEpQvp_nNO1in...,gAAAAABpLrCMtj98pR40ir08dPPe10vTiJcJMu8eVpDDLy...
4,5,Eva,eva@example.com,Engineering,gAAAAABpLrCMRSI_KPTpK02SBff78xRmhqPPoE-_0r0jOR...,gAAAAABpLrCMau4X6IX1fNkwVhznV7ncx_dOuB7sQqq2Px...


## Test with Read-only Role

In [14]:
key = get_key(HR)
print("Retrieved key for HR:", key)
file = retrieve_data(BUCKET_NAME, SALARY_ENCRYPTED_FILE_NAME)
decrypt_salary_with_key(key["fernet_key"], file, "Salary")

Retrieved key for HR: {'fernet_key': 'U1eIY6p4bKjOaMycX1VyMshD0tRmfWqC7xJ0MMT8oO0='}

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.


,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,55000,gAAAAABpLrCMtGSy6lBrX4Ge4B_tADTvGpQkWCZ9MzGOi2...
1,2,Bob,bob@example.com,Engineering,72000,gAAAAABpLrCMqaffpffNlnWpyXigdcRQs35FD0evnJoPoV...
2,3,Charlie,charlie@example.com,Marketing,63000,gAAAAABpLrCMIeIebdmsd7i0hXysvI9OfMakQ_aSQys72G...
3,4,David,david@example.com,Finance,80000,gAAAAABpLrCMtj98pR40ir08dPPe10vTiJcJMu8eVpDDLy...
4,5,Eva,eva@example.com,Engineering,75000,gAAAAABpLrCMau4X6IX1fNkwVhznV7ncx_dOuB7sQqq2Px...
5,6,Frank,frank@example.com,HR,50679,gAAAAABpLrCM8nF4MU55PAZKcbCsm0iq_BmbgY44a3RTsw...
6,7,Grace,grace@example.com,Sales,98115,gAAAAABpLrCMt2n2Qz8PUvMNzDze3xA8catvbqhXacJ4lw...
7,8,Hannah,hannah@example.com,Finance,87984,gAAAAABpLrCM_JVNJnll5BE_4cEEc5KojU2FBrdM0usQe8...
8,9,Ian,ian@example.com,Marketing,118777,gAAAAABpLrCMT0zpUl-kD3td_FCJOqDyNQHajggsZmN06v...
9,10,Julia,julia@example.com,Sales,69355,gAAAAABpLrCMgKWCdV9nVhIQ1PkL4mDMt3Ih6f7zpcqmAD...


## Test with Admin Role

In [15]:
key = get_key(ADMIN)
print("Retrieved key for HR:", key)
file = retrieve_data(BUCKET_NAME, SALARY_ENCRYPTED_FILE_NAME)
df = decrypt_salary_with_key(key["fernet_key"], file, "Salary")
decrypt_password_with_key(key["admin_key"], df, "Password")

Retrieved key for HR: {'fernet_key': 'U1eIY6p4bKjOaMycX1VyMshD0tRmfWqC7xJ0MMT8oO0=', 'admin_key': None}

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.
No key provided, cannot decrypt.


,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,55000,gAAAAABpLrCMtGSy6lBrX4Ge4B_tADTvGpQkWCZ9MzGOi2...
1,2,Bob,bob@example.com,Engineering,72000,gAAAAABpLrCMqaffpffNlnWpyXigdcRQs35FD0evnJoPoV...
2,3,Charlie,charlie@example.com,Marketing,63000,gAAAAABpLrCMIeIebdmsd7i0hXysvI9OfMakQ_aSQys72G...
3,4,David,david@example.com,Finance,80000,gAAAAABpLrCMtj98pR40ir08dPPe10vTiJcJMu8eVpDDLy...
4,5,Eva,eva@example.com,Engineering,75000,gAAAAABpLrCMau4X6IX1fNkwVhznV7ncx_dOuB7sQqq2Px...
5,6,Frank,frank@example.com,HR,50679,gAAAAABpLrCM8nF4MU55PAZKcbCsm0iq_BmbgY44a3RTsw...
6,7,Grace,grace@example.com,Sales,98115,gAAAAABpLrCMt2n2Qz8PUvMNzDze3xA8catvbqhXacJ4lw...
7,8,Hannah,hannah@example.com,Finance,87984,gAAAAABpLrCM_JVNJnll5BE_4cEEc5KojU2FBrdM0usQe8...
8,9,Ian,ian@example.com,Marketing,118777,gAAAAABpLrCMT0zpUl-kD3td_FCJOqDyNQHajggsZmN06v...
9,10,Julia,julia@example.com,Sales,69355,gAAAAABpLrCMgKWCdV9nVhIQ1PkL4mDMt3Ih6f7zpcqmAD...


In [ ]:
key = get_key(HR)
print("Retrieved key for HR:", key)
file = retrieve_data(BUCKET_NAME, SALARY_ENCRYPTED_FILE_NAME)
decrypt_salary_with_key(key["fernet_key"], file, "Salary")


Retrieved key for HR: {'fernet_key': 'U1eIY6p4bKjOaMycX1VyMshD0tRmfWqC7xJ0MMT8oO0='}

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.


,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,55000,gAAAAABpLrCMtGSy6lBrX4Ge4B_tADTvGpQkWCZ9MzGOi2...
1,2,Bob,bob@example.com,Engineering,72000,gAAAAABpLrCMqaffpffNlnWpyXigdcRQs35FD0evnJoPoV...
2,3,Charlie,charlie@example.com,Marketing,63000,gAAAAABpLrCMIeIebdmsd7i0hXysvI9OfMakQ_aSQys72G...
3,4,David,david@example.com,Finance,80000,gAAAAABpLrCMtj98pR40ir08dPPe10vTiJcJMu8eVpDDLy...
4,5,Eva,eva@example.com,Engineering,75000,gAAAAABpLrCMau4X6IX1fNkwVhznV7ncx_dOuB7sQqq2Px...
5,6,Frank,frank@example.com,HR,50679,gAAAAABpLrCM8nF4MU55PAZKcbCsm0iq_BmbgY44a3RTsw...
6,7,Grace,grace@example.com,Sales,98115,gAAAAABpLrCMt2n2Qz8PUvMNzDze3xA8catvbqhXacJ4lw...
7,8,Hannah,hannah@example.com,Finance,87984,gAAAAABpLrCM_JVNJnll5BE_4cEEc5KojU2FBrdM0usQe8...
8,9,Ian,ian@example.com,Marketing,118777,gAAAAABpLrCMT0zpUl-kD3td_FCJOqDyNQHajggsZmN06v...
9,10,Julia,julia@example.com,Sales,69355,gAAAAABpLrCMgKWCdV9nVhIQ1PkL4mDMt3Ih6f7zpcqmAD...
